In [1]:
import requests 
import json 
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
import pandas as pd 
import numpy as np 
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 50)

In [3]:
df = pd.read_csv("~/Downloads/Kickstarter_2021-04-15T03_20_08_451Z/Kickstarter.csv")

In [4]:
def proj_url(df, index):
    tmp = df.iloc[index]
    # is stored as a json type 
    url = json.loads(tmp['urls'])['web']['project']
    return url 

In [5]:
tmp = df.iloc[0]
tmp_url = json.loads(tmp['urls'])['web']['project']

In [6]:
def load_text_url(url):
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, features="html.parser")
    # kill all script and style elements
    return soup 
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    # get text
    text = soup.get_text()
    return soup.find_all(text=True)

In [7]:
driver = webdriver.Chrome("/Users/benji/Downloads/chromedriver")

In [8]:
def extract_proj_descrip(url):
    # i think descriptions are pulled up w/ some java script bs so we need to load in driver
    driver.get(url)
    html = driver.page_source
    # load html 
    soup = BeautifulSoup(html)
    proj = soup.find("section", {"class": "project-content"})
    return proj, proj.find_all(text=True)

In [35]:
p = extract_proj_descrip(tmp_url)

In [9]:
p, ptxt = extract_proj_descrip(proj_url(df, 2))

In [14]:
for i in range(2, 5):
    p, ptxt = extract_proj_descrip(proj_url(df, i))
    print(shorten_list(ptxt))

['Story', 'Here at ', 'Dreamcatcher Comics', ' we are a start up company about to produce issue one of our comic The Gatekeepers.', 'We are looking at creating a rollerball game and other games/apps. We are currently looking for funding to cover such things as Software etc.\xa0', 'This Game will be a puzzle game using our very own Hunter as your first main character with more unlockable characters thoughout the game. at first your main challenge will be battling though the many levels and defeating terrain. Afterwards your face with defeating the Alptraum, The Gatekeepers most feared enimies. they will cause you all kinds of trouble by setting you time limits and throwing obsticles in your way.', 'Our main aim for this game will be to have both joystick like controls and also Tilt controls for smart phones. We will be bringing this Game to both android and iphone and all relating tablets. We also hope to bring out a pc version of the game to allowing all kinds of gamers to play.', 'We 

IndexError: index -1 is out of bounds for axis 0 with size 0

In [84]:
[w for w in ptxt if w != '\n']

['\nSpawns of the Great Rot - Printable stl 3d miniatures\n',
 '\nSvendborg, Denmark\n',
 '\nTabletop Games\n',
 'DKK 104,176',
 '\npledged of ',
 'DKK 7,500',
 ' goal\n',
 '\n454\n',
 '\nbackers\n',
 'Support',
 'Select this reward',
 '\nPledge ',
 'DKK 150',
 ' or more\n',
 '\nAbout ',
 'US$ 25',
 '\nEarly Bird - Personal\n',
 'Discount!\n',
 '- All STL models in the campaign including stretch goals.\n',
 '- Presupported!\n',
 '- For personal use only.',
 'Less',
 'Estimated delivery',
 'Jan 2021',
 '\n99 backers\n',
 '\nLimited (1 left of 100)\n',
 'Pledge amount',
 'DKK',
 '\nContinue\n',
 '\nOther payment options\n',
 "\nBy pledging you agree to Kickstarter's ",
 'Terms of Use',
 ', ',
 'Privacy Policy',
 ', and ',
 'Cookie Policy',
 '.\n\n',
 '\nKickstarter is not a store.\n',
 "\nIt's a way to bring creative projects to life.\n",
 'Learn more about accountability.',
 'Select this reward',
 '\nPledge ',
 'DKK 200',
 ' or more\n',
 '\nAbout ',
 '$33',
 '\nPersonal\n',
 '- All STL 

In [52]:
tst = [w for w in ptxt if w != '\n']

In [62]:
tst

['\nSpawns of the Great Rot - Printable stl 3d miniatures\n',
 '\nSvendborg, Denmark\n',
 '\nTabletop Games\n',
 'DKK 104,176',
 '\npledged of ',
 'DKK 7,500',
 ' goal\n',
 '\n454\n',
 '\nbackers\n',
 'Story',
 'Risks',
 'Environmental commitments',
 'Story',
 "Before the great rot, death was permanent. Unfortunately, death for those who were affected by the Soul Rot wasn't the end.",
 'The Soul Rot forced the soul to live on after the body gave up, condemning the soul to live forever in the state it left the body. These poor souls are known as Geists.',
 'Welcome to the first campaign by Fleshcraft studio!',
 '\n \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Working on a scary campaign for your DnD group? Or wanna expand the looks of your undead armies? Well, “unfortunately” for you, the ghosts have come to haunt you! In this campaign, you will receive a set of terrifying digital 3D ghosts in the .stl format. ',
 '\n \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\x

In [12]:
def shorten_list(wrds):
    # words signal end of project description 
    beg = np.where(np.array(wrds)=='Story')[0][-1]
    end1 = np.where(np.array(wrds)=='Check out the FAQ')[0][0]
    end2 = np.where(np.array(wrds)=='Report this project to Kickstarter')[0][0]
    end3 = np.where(np.array(wrds)=='Select this reward')[0][0]
    min_ind = min(end1, end2, end3)
    return wrds[beg:min_ind]

In [13]:
shorten_list(ptxt)

['Story',
 'Here at ',
 'Dreamcatcher Comics',
 ' we are a start up company about to produce issue one of our comic The Gatekeepers.',
 'We are looking at creating a rollerball game and other games/apps. We are currently looking for funding to cover such things as Software etc.\xa0',
 'This Game will be a puzzle game using our very own Hunter as your first main character with more unlockable characters thoughout the game. at first your main challenge will be battling though the many levels and defeating terrain. Afterwards your face with defeating the Alptraum, The Gatekeepers most feared enimies. they will cause you all kinds of trouble by setting you time limits and throwing obsticles in your way.',
 'Our main aim for this game will be to have both joystick like controls and also Tilt controls for smart phones. We will be bringing this Game to both android and iphone and all relating tablets. We also hope to bring out a pc version of the game to allowing all kinds of gamers to play.'